In [1]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from pyearth import Earth
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.model_selection import StratifiedKFold

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
def apply_smote(df):
    df.reset_index(drop=True,inplace=True)
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

In [3]:
def load_product_data(project):
    # Processing Product files
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    cols = understand_df.columns.tolist()
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    df = understand_df
    cols = df.columns.tolist()
    cols.remove('Bugs')
    cols.append('Bugs')
    _df = df
    df = df[cols]
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    return X,y,_df

In [4]:
def load_process_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    understand_df = pd.read_csv(understand_path)
    commits = understand_df.commit_hash.unique()
    commit_guru_file_level_path = 'data/commit_guru_file_level/' + project + '_file.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    commit_guru_file_level_df = pd.read_csv(commit_guru_file_level_path)
    commit_guru_file_level_df['commit_hash'] = commit_guru_file_level_df.commit_hash.str.strip('"')
    commit_guru_df = pd.read_csv(commit_guru_path)
    commit_guru_df = commit_guru_df[['commit_hash','contains_bug']]
    values = {'contains_bug': False}
    commit_guru_df = commit_guru_df.fillna(values)
#     commit_guru_df = commit_guru_df[commit_guru_df.commit_hash.isin(commits)]
    df = commit_guru_file_level_df.merge(commit_guru_df,on='commit_hash') 
    df.rename(columns={"contains_bug": "Bugs"},inplace=True)
    df = df[df['file_name'].str.contains('.java')]
    df = df.drop(['commit_hash','file_name'],axis = 1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = df.drop_duplicates()
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    return X,y

In [5]:
def load_both_data(project,metric):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    cols = understand_df.columns.tolist()
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    understand_df['Name'] = understand_df.Name.str.rsplit('.',1).str[1]
    
    commit_guru_file_level_path = 'data/commit_guru_file_level/' + project + '_file.csv'
    commit_guru_file_level_df = pd.read_csv(commit_guru_file_level_path)
    commit_guru_file_level_df['commit_hash'] = commit_guru_file_level_df.commit_hash.str.strip('"')
    commit_guru_file_level_df = commit_guru_file_level_df[commit_guru_file_level_df['file_name'].str.contains('.java')]
    commit_guru_file_level_df['Name'] = commit_guru_file_level_df.file_name.str.rsplit('/',1).str[1].str.split('.').str[0].str.replace('/','.')
    commit_guru_file_level_df = commit_guru_file_level_df.drop('file_name',axis = 1)
    
    
    df = understand_df.merge(commit_guru_file_level_df,how='left',on=['commit_hash','Name'])
    
    
    cols = df.columns.tolist()
    cols.remove('Bugs')
    cols.append('Bugs')
    df = df[cols]
    
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
#     df.dropna(inplace=True)
    df = df.drop_duplicates()
    df.reset_index(drop=True, inplace=True)
    
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    imp_mean = IterativeImputer(random_state=0)
    X = imp_mean.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    
    if metric == 'process':
        X = X[['la', 'ld', 'lt', 'age', 'ndev', 'nuc', 'ns', 'exp', 'sexp', 'rexp', 'nd']]
    elif metric == 'product':
        X = X.drop(['la', 'ld', 'lt', 'age', 'ndev', 'nuc', 'ns', 'exp', 'sexp', 'rexp', 'nd'],axis = 1)
    else:
        X = X
    return X,y

In [38]:
def run_self(project,metric):
    X,y = load_both_data(project,metric)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    if metric == 'process':
        loc = X_test['la'] + X_test['lt']
    elif metric == 'product':
        loc = X_test.CountLineCode
    else:
        loc = X_test['la'] + X_test['lt']
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = RandomForestClassifier()
    clf.fit(X_train,y_train)
    importance = clf.feature_importances_
    print(len(importance))
    predicted = clf.predict(X_test)
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [6]:
def run_self_k(project,metric):
    precision = []
    recall = []
    pf = []
    f1 = []
    g_score = []
    auc = []
    pci_20 = []
    ifa = []
    importance = []
    X,y = load_both_data(project,metric)
    for _ in range(5):
        skf = StratifiedKFold(n_splits=5)
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X.loc[train_index], X.loc[test_index]
            y_train, y_test = y.loc[train_index], y.loc[test_index]
            if metric == 'process':
                loc = X_test['la'] + X_test['lt']
            elif metric == 'product':
                loc = X_test.CountLineCode
            else:
                loc = X_test['la'] + X_test['lt']
            df_smote = pd.concat([X_train,y_train],axis = 1)
            df_smote = apply_smote(df_smote)
            y_train = df_smote.Bugs
            X_train = df_smote.drop('Bugs',axis = 1)
            clf = RandomForestClassifier()
            clf.fit(X_train,y_train)
            importance = clf.feature_importances_
            predicted = clf.predict(X_test)
            abcd = metrices.measures(y_test,predicted,loc)
            pf.append(abcd.get_pf())
            recall.append(abcd.calculate_recall())
            precision.append(abcd.calculate_precision())
            f1.append(abcd.calculate_f1_score())
            g_score.append(abcd.get_g_score())
            pci_20.append(abcd.get_pci_20())
            ifa.append(abcd.get_ifa())
            try:
                auc.append(roc_auc_score(y_test, predicted))
            except:
                auc.append(0)
#             print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [7]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

In [8]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
pci_20_list = {}
ifa_list = {}
featue_importance = {}
for project in projects[150:]:
    try:
        if project == '.DS_Store':
            continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance = run_self_k(project,'product')
        recall_list[project] = recall
        precision_list[project] = precision
        pf_list[project] = pf
        f1_list[project] = f1
        g_list[project] = g_score
        auc_list[project] = auc
        pci_20_list[project] = pci_20
        ifa_list[project] = ifa
        featue_importance[project] = importance
    except Exception as e:
        print(e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
final_result['pci_20'] = pci_20_list
final_result['ifa'] = ifa_list
final_result['featue_importance'] = featue_importance

+++++++++++++++++   spring-xd-samples  +++++++++++++++++
+++++++++++++++++   environment-dashboard  +++++++++++++++++
+++++++++++++++++   plantuml  +++++++++++++++++
+++++++++++++++++   egads  +++++++++++++++++
+++++++++++++++++   dockerfile-maven  +++++++++++++++++
+++++++++++++++++   android-pdfview  +++++++++++++++++
+++++++++++++++++   rebound  +++++++++++++++++
+++++++++++++++++   react-native-background-timer  +++++++++++++++++
+++++++++++++++++   android-rss  +++++++++++++++++
+++++++++++++++++   logback-kafka-appender  +++++++++++++++++
+++++++++++++++++   Droppy  +++++++++++++++++
+++++++++++++++++   RxActivityResult  +++++++++++++++++
+++++++++++++++++   smart-show  +++++++++++++++++
+++++++++++++++++   PatternLock  +++++++++++++++++
+++++++++++++++++   RxHttpUtils  +++++++++++++++++
+++++++++++++++++   android-transcoder  +++++++++++++++++
+++++++++++++++++   pcollections  +++++++++++++++++
+++++++++++++++++   libsignal-protocol-java  +++++++++++++++++
+++++++++++++++++   ES

+++++++++++++++++   cordova-plugin-googleplus  +++++++++++++++++
+++++++++++++++++   jacoco  +++++++++++++++++
+++++++++++++++++   FloatingActionButton  +++++++++++++++++
+++++++++++++++++   pinned-section-listview  +++++++++++++++++
Shape of passed values is (61, 45), indices imply (61, 61)
+++++++++++++++++   dynamic-load-apk  +++++++++++++++++
Shape of passed values is (243, 45), indices imply (243, 61)
+++++++++++++++++   elasticsearch-analysis-ik  +++++++++++++++++
Shape of passed values is (149, 45), indices imply (149, 61)
+++++++++++++++++   jersey  +++++++++++++++++
Shape of passed values is (5148, 45), indices imply (5148, 56)
+++++++++++++++++   corretto-8  +++++++++++++++++
+++++++++++++++++   CustomBottomSheetBehavior  +++++++++++++++++
Shape of passed values is (82, 45), indices imply (82, 56)
+++++++++++++++++   Pydev  +++++++++++++++++
+++++++++++++++++   MultiImagePicker  +++++++++++++++++
+++++++++++++++++   spring-test-dbunit  +++++++++++++++++
+++++++++++++++++   An

+++++++++++++++++   snackbar  +++++++++++++++++
+++++++++++++++++   react-native-baidu-map  +++++++++++++++++
+++++++++++++++++   kafka-connect-elasticsearch  +++++++++++++++++
+++++++++++++++++   neurolab-android  +++++++++++++++++
+++++++++++++++++   soul  +++++++++++++++++
+++++++++++++++++   jmx_exporter  +++++++++++++++++
+++++++++++++++++   RecyclerViewPager  +++++++++++++++++
+++++++++++++++++   ChatKit  +++++++++++++++++
+++++++++++++++++   jvm-sandbox  +++++++++++++++++
+++++++++++++++++   android-crop  +++++++++++++++++
+++++++++++++++++   quartz  +++++++++++++++++
+++++++++++++++++   jenkins-gitlab-merge-request-builder-plugin  +++++++++++++++++
Shape of passed values is (169, 45), indices imply (169, 56)
+++++++++++++++++   TextJustify-Android  +++++++++++++++++
+++++++++++++++++   PhotoPicker  +++++++++++++++++
+++++++++++++++++   jjwt  +++++++++++++++++
+++++++++++++++++   Android-Week-View  +++++++++++++++++
+++++++++++++++++   rejoiner  +++++++++++++++++
+++++++++++++++

+++++++++++++++++   iceberg  +++++++++++++++++
+++++++++++++++++   material-remixer-android  +++++++++++++++++
+++++++++++++++++   jackson-annotations  +++++++++++++++++
+++++++++++++++++   sofa-ark  +++++++++++++++++
+++++++++++++++++   LeetCode-Sol-Res  +++++++++++++++++
+++++++++++++++++   NullAway  +++++++++++++++++
+++++++++++++++++   easymock  +++++++++++++++++
+++++++++++++++++   dialogflow-android-client  +++++++++++++++++
+++++++++++++++++   mbassador  +++++++++++++++++
+++++++++++++++++   TokenAutoComplete  +++++++++++++++++
+++++++++++++++++   mysql-binlog-connector-java  +++++++++++++++++
+++++++++++++++++   airpal  +++++++++++++++++
+++++++++++++++++   cordova-plugin-background-mode  +++++++++++++++++
+++++++++++++++++   awaitility  +++++++++++++++++
+++++++++++++++++   amazon-kinesis-client  +++++++++++++++++
+++++++++++++++++   react-native-share  +++++++++++++++++
+++++++++++++++++   jhipster-registry  +++++++++++++++++
+++++++++++++++++   ShowcaseView  ++++++++++++++++

+++++++++++++++++   Android-Debug-Database  +++++++++++++++++
+++++++++++++++++   retrolambda  +++++++++++++++++
+++++++++++++++++   java  +++++++++++++++++
Shape of passed values is (333, 45), indices imply (333, 56)
+++++++++++++++++   TelegramBots  +++++++++++++++++
Shape of passed values is (1446, 45), indices imply (1446, 56)
+++++++++++++++++   liferay-docs  +++++++++++++++++
+++++++++++++++++   bootique  +++++++++++++++++
+++++++++++++++++   secor  +++++++++++++++++
+++++++++++++++++   BottomSheetPickers  +++++++++++++++++
+++++++++++++++++   android-advancedrecyclerview  +++++++++++++++++
+++++++++++++++++   android-smsmms  +++++++++++++++++
+++++++++++++++++   swiftp  +++++++++++++++++
+++++++++++++++++   karyon  +++++++++++++++++
+++++++++++++++++   jasmine-maven-plugin  +++++++++++++++++
+++++++++++++++++   togglz  +++++++++++++++++
+++++++++++++++++   GravityBox  +++++++++++++++++
Shape of passed values is (1459, 45), indices imply (1459, 56)
+++++++++++++++++   JASONETTE-A

+++++++++++++++++   s3proxy  +++++++++++++++++
+++++++++++++++++   metacat  +++++++++++++++++
+++++++++++++++++   J2ME-Loader  +++++++++++++++++
+++++++++++++++++   react-native-iap  +++++++++++++++++
+++++++++++++++++   leshan  +++++++++++++++++
+++++++++++++++++   cucumber-reporting  +++++++++++++++++
+++++++++++++++++   unfolding  +++++++++++++++++
+++++++++++++++++   redis-manager  +++++++++++++++++
+++++++++++++++++   java-driver  +++++++++++++++++
Shape of passed values is (5433, 45), indices imply (5433, 56)
+++++++++++++++++   tx-lcn  +++++++++++++++++
+++++++++++++++++   SecurityShepherd  +++++++++++++++++
+++++++++++++++++   subclipse  +++++++++++++++++
+++++++++++++++++   couchdb-lucene  +++++++++++++++++
+++++++++++++++++   programming  +++++++++++++++++
Shape of passed values is (211, 45), indices imply (211, 56)
+++++++++++++++++   OpenRTS  +++++++++++++++++
+++++++++++++++++   spectator  +++++++++++++++++
Shape of passed values is (1895, 45), indices imply (1895, 56)
+++

+++++++++++++++++   boon  +++++++++++++++++
+++++++++++++++++   spring-data-solr  +++++++++++++++++
+++++++++++++++++   javassist  +++++++++++++++++
Shape of passed values is (3196, 45), indices imply (3196, 56)
+++++++++++++++++   uhabits  +++++++++++++++++
+++++++++++++++++   spock  +++++++++++++++++
+++++++++++++++++   PageTurner  +++++++++++++++++
+++++++++++++++++   heroic  +++++++++++++++++
+++++++++++++++++   univocity-parsers  +++++++++++++++++
+++++++++++++++++   twilio-java  +++++++++++++++++
+++++++++++++++++   TrebleShot  +++++++++++++++++
+++++++++++++++++   interview  +++++++++++++++++
+++++++++++++++++   jsprit  +++++++++++++++++
+++++++++++++++++   focus-android  +++++++++++++++++
+++++++++++++++++   spring-cloud-commons  +++++++++++++++++
Shape of passed values is (1895, 45), indices imply (1895, 56)
+++++++++++++++++   spring-hateoas  +++++++++++++++++
Shape of passed values is (2601, 45), indices imply (2601, 56)
+++++++++++++++++   flinkx  +++++++++++++++++
++++++++

In [9]:
with open('results/Performance/commit_guru_file_specific/product_700_rf_5_fold_5_repeat.pkl', 'wb') as handle:
    pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)